In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("video recommendation")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

In [2]:
inputPath="/home/jatin/Documents/spark-notebook-0.6.2-scala-2.11.7-spark-1.6.0-hadoop-2.2.0-with-hive-with-parquet/videoMetaData3csv/videoMetaDataInCsvFormat"

In [3]:
englishStopWords = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]

In [4]:
#Reading file containing video meta data to make initial RDD
readFile = sc.textFile(inputPath).cache()
readFile.count()

59800

In [5]:
readFile.take(5)

[u'0_04xqwyas,\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"',
 u'0_0m8u8g4c,\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"',
 u'0_0pd5gia9,\'MythBusters\': A MythBusting Lightsaber Duel,Jamie and Adam put the myth of higher ground to the test in this epic lightsaber battle.,"reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, lightsaber dueling"',
 u'0_0uwz3tkl,Behind the Scenes - Lucky Influencers: What Was Your Fashion First?,"5 fashion industry influencers, Va$shtie Kola, Lily Kwong, Amirah Kassem,

In [6]:
# Converting string into (ID,remainingString) format
def formatInput(item):
     name = item.split(",")[0]
     return (name,item.replace(name+",",""))

nameFromCsv = readFile.map(formatInput)
#                                name = item.split(",")(0)
#                               (name,item.replace(name+",",""))
#                               }

In [7]:
nameFromCsv.take(2)

[(u'0_04xqwyas',
  u'\'Edge of Alaska\': Keeping a Frontier Town Remote,All it takes is one big attraction to change everything. Will McCarthy be next?,"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska"'),
 (u'0_0m8u8g4c',
  u'\'MythBusters\': Shark Feeding Frenzy Defense,"While testing the repelling abilities of ""essence of dead shark"", Jamie hopes his beret will serve as more than just a fashion statement.","reality tv series, mythbusters, jamie hyneman, adam savage, urban legends, shark myths, feeding frenzy, mythbusters shark tests, shark armor"')]

In [8]:
reserveWords = sc.broadcast(englishStopWords)

In [9]:
import re
def bagOfWords(item):
    regexSpecialChars = """[^a-zA-Z0-9\s]"""
    noSymbolsLine = re.sub(regexSpecialChars, " ",item)
    stopWords = reserveWords.value
    listOfWords = noSymbolsLine.split()
    nonEmptyWords = filter(lambda x :x != "",listOfWords)
    removeReserveWords = []
    for word in nonEmptyWords:
        if word not in stopWords:
            removeReserveWords.append(word.lower())
    return removeReserveWords

In [10]:
bow = bagOfWords("'Edge of Alaska': Keeping a Frontier 1232 Town  @ #Remote,All it takes is () *%* one big attraction to change everything. Will McCarthy be next?,\"reality tv series, edge of alaska, mccarthy alaska, frontier town, off grid living, takeetna alaska\"")
print bow

['edge', 'alaska', 'keeping', 'frontier', '1232', 'town', 'remote', 'all', 'takes', 'big', 'attraction', 'change', 'will', 'mccarthy', 'reality', 'tv', 'series', 'edge', 'alaska', 'mccarthy', 'alaska', 'frontier', 'town', 'grid', 'living', 'takeetna', 'alaska']


In [11]:
cleandData = nameFromCsv.mapValues(bagOfWords).cache()

In [12]:
# Cross Checking count of data after cleaning 
numDocs = cleandData.count()

In [13]:
print numDocs

59800


In [14]:
readFile.unpersist()
from collections import Counter
def calculateTermFreq(item):
    return Counter(item).items()

In [15]:
it = calculateTermFreq(bow)

In [16]:
print it

[('town', 2), ('living', 1), ('keeping', 1), ('remote', 1), ('takes', 1), ('1232', 1), ('frontier', 2), ('tv', 1), ('big', 1), ('alaska', 4), ('mccarthy', 2), ('will', 1), ('attraction', 1), ('edge', 2), ('grid', 1), ('all', 1), ('series', 1), ('takeetna', 1), ('reality', 1), ('change', 1)]


In [30]:
termDocumentFrequencies = cleandData.mapValues(calculateTermFreq)

In [32]:
termDocumentFrequencies.take(1)#[0][0]

[(u'0_04xqwyas',
  [(u'town', 2),
   (u'living', 1),
   (u'keeping', 1),
   (u'remote', 1),
   (u'takes', 1),
   (u'frontier', 2),
   (u'tv', 1),
   (u'big', 1),
   (u'alaska', 4),
   (u'mccarthy', 2),
   (u'will', 1),
   (u'attraction', 1),
   (u'edge', 2),
   (u'grid', 1),
   (u'all', 1),
   (u'series', 1),
   (u'takeetna', 1),
   (u'reality', 1),
   (u'change', 1)])]

In [19]:
# Assigning unique ids to each term in the document
idDocs = termDocumentFrequencies.map(lambda x: x[0]).zipWithUniqueId().collectAsMap()#.toMap

In [20]:
type(idDocs)

dict

In [21]:
docIds = dict((v,k) for k,v in idDocs.iteritems())
# docIds.take(2)

In [22]:
docIds[0]

u'0_04xqwyas'

In [23]:
docVect = termDocumentFrequencies.flatMapValues(lambda x:x).values().reduceByKey(lambda x,y:x + y).cache()

In [24]:
cleandData.unpersist()

PythonRDD[5] at RDD at PythonRDD.scala:48

In [25]:
docVect.count()

34629

In [37]:
docVect.take(10)[2][0]

u'abenteuerliche'

In [41]:
corpusVocab = docVect.filter(lambda item: item[1] > 5 and len(item[0]) > 3).collect()

In [55]:
# FUNCTION TO COMPUTE INVERSE DCOUMENT FREQUENCIES
from math import log1p
def inverseDocumentFrequencies(docFreqs, numDocs):
    return map( lambda tup : (tup[0], log1p(float(numDocs) / tup[1])), docFreqs)


#Computing inverse document frequencies 'idfs' from document frequencies
idfs = inverseDocumentFrequencies(corpusVocab, numDocs)

idfsMap = dict(idfs)

# /* Broadcasting 'idfs' across nodes of cluster*/
bidfs = sc.broadcast(idfsMap)

# /* Collecting all the terms after filtering (terms, df) pairs*/
vocabulary = idfsMap.keys().zipWithIndex()

# /* Broadcasting vocabulary across all nodes of clusters*/
# val termList: Broadcast[Predef.Map[String, Int]] = sc.broadcast(vocabulary)

# /* Terms are associated with the ids as shown (id, term) */
# val termIds: Predef.Map[Int, String] = vocabulary.map(_.swap).toMap 


AttributeError: 'list' object has no attribute 'zipWithIndex'

In [53]:
type(idfs[1])

tuple